In [1]:
import tensorflow as tf
from distutils.version import LooseVersion
import warnings

import keras
from keras import models
from keras import layers as l

import numpy as np
import matplotlib.pyplot as plt

import random
import sys

assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Using TensorFlow backend.


TensorFlow Version: 2.2.0
Default GPU Device: /device:GPU:0


In [2]:
path = keras.utils.get_file(
    'nietzsche.txt'
    , origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt'
)
text = open(path).read().lower()

In [3]:
len(text)

600901

In [4]:
maxlen = 60
step = 3

sentences = []
next_chars = []

for x in range(0, len(text) - maxlen, step):
    sentences.append(text[x: x + maxlen])
    next_chars.append(text[x + maxlen])

print(len(sentences))
print(len(next_chars))

200281
200281


In [5]:
chars = sorted(list(set(text)))
char_indices = dict((char, chars.index(char)) for char in chars)
len(chars) # unique characters

59

In [6]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence, in enumerate(sentences):
    for j, char in enumerate(sentence):
        x[i, j, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [7]:
model = models.Sequential()
model.add(l.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(l.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [8]:
def sample(preds, temp=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temp
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

In [9]:
# train
for epoch in range(1, 60):
    print(f'Epoch: {epoch}\n')
    model.fit(x, y, batch_size=128, epochs=1)
    start_i = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_i: start_i + maxlen]
    print(f'Seed: {generated_text}\n')
    for temp in [0.2, 0.5, 1.0, 1.2]:
        print(f'Temperature: {temp}\n')
        sys.stdout.write(generated_text)
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))             
            for t, char in enumerate(generated_text):               
                sampled[0, t, char_indices[char]] = 1.              
            preds = model.predict(sampled, verbose=0)[0]            
            next_index = sample(preds, temp)                 
            next_char = chars[next_index]                           
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)

Epoch: 1

1565/1565 [==============================] - 13s 8ms/step - loss: 1.9749

C:\Users\Bill\anaconda3\envs\DeepLearning\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
model.save("LSTM.h5")
